# MyMedicalGPT - Colab Training Pipeline

这个Notebook帮助你在Google Colab上训练自己的大语言模型

**使用步骤**：
1. 配置Colab GPU: `代码执行程序 → 更改运行时类型 → GPU → T4`
2. 依次运行下面的代码块
3. 训练完成后下载模型权重

**训练阶段**：
- Stage 1: 预训练 (PT) - 可选
- Stage 2: 有监督微调 (SFT) - 必须
- Stage 3: 直接偏好优化 (DPO) - 可选

**预计时间**：
- 仅SFT: 20-30分钟
- 完整流程: 1-2小时

---

## 📦 Part 1: 环境配置

首次运行需要安装依赖，约需5分钟

In [ ]:
# 检查GPU
!nvidia-smi

In [ ]:
# 方式1: 从GitHub克隆项目（推荐）
# 修改为你自己的仓库地址
!git clone https://github.com/DemonRain7/MyMedicalGPT.git
%cd MyMedicalGPT

In [ ]:
# 方式2: 从Google Drive加载（如果你已经上传了项目）
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/MyMedicalGPT

In [ ]:
# 安装依赖
!pip install -q transformers>=4.49.0 datasets peft trl accelerate loguru tensorboard

print("✓ 依赖安装完成！")

In [ ]:
# 查看项目结构
!ls -lh

## 📊 Part 2: 数据准备

检查数据集是否存在

In [ ]:
# 查看示例数据
print("=== 预训练数据 ===")
!ls -lh data/pretrain/

print("\n=== SFT数据 ===")
!ls -lh data/finetune/

print("\n=== DPO数据 ===")
!ls -lh data/reward/

In [ ]:
# 查看SFT数据格式示例
!head -n 2 data/finetune/medical_sft_1K_format.jsonl

### 📤 上传自己的数据（可选）

如果你想使用自己的数据，运行下面的代码块上传文件

In [ ]:
# 上传自己的训练数据
from google.colab import files

print("请上传你的训练数据（.jsonl格式，ShareGPT格式）")
uploaded = files.upload()

# 移动到data目录
for filename in uploaded.keys():
    !mv {filename} data/finetune/
    print(f"✓ {filename} 已上传到 data/finetune/")

## 🚀 Part 3: 模型训练

### 选择训练方案：
- **方案A**: 仅SFT（推荐新手，快速）
- **方案B**: PT + SFT（有领域数据）
- **方案C**: PT + SFT + DPO（完整流程）

---

### 🎯 方案A: 仅SFT训练（推荐）

直接在基座模型上做指令微调，最简单快速

In [ ]:
# Stage 2: SFT训练
# 使用Qwen2.5-0.5B作为基座模型（小而快）
# 如果想用更大的模型，可以改成 Qwen/Qwen2.5-1.5B 或 Qwen/Qwen2.5-3B

!python supervised_finetuning.py \
    --model_name_or_path Qwen/Qwen2.5-0.5B-Instruct \
    --train_file_dir ./data/finetune \
    --validation_file_dir ./data/finetune \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --template_name qwen \
    --bf16 \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --num_train_epochs 3 \
    --learning_rate 2e-5 \
    --warmup_ratio 0.05 \
    --weight_decay 0.05 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --eval_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --output_dir outputs-sft \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype bfloat16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

In [ ]:
# 查看训练结果
!ls -lh outputs-sft/

In [ ]:
# 合并LoRA权重到基座模型（可选，推荐直接使用LoRA）
!python merge_peft_adapter.py \
    --base_model Qwen/Qwen2.5-0.5B-Instruct \
    --lora_model outputs-sft \
    --output_dir merged-sft

print("✓ 模型合并完成！")

### 🔄 方案B: PT + SFT（有领域数据时使用）

先在领域文本上预训练，再做指令微调

In [ ]:
# Stage 1: 预训练
!python pretraining.py \
    --model_name_or_path Qwen/Qwen2.5-0.5B \
    --train_file_dir ./data/pretrain \
    --validation_file_dir ./data/pretrain \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 3 \
    --do_train \
    --do_eval \
    --use_peft True \
    --seed 42 \
    --bf16 \
    --max_train_samples 20000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-4 \
    --warmup_ratio 0.05 \
    --weight_decay 0.01 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --eval_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --block_size 128 \
    --output_dir outputs-pt \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype bfloat16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

In [ ]:
# 合并PT模型
!python merge_peft_adapter.py \
    --base_model Qwen/Qwen2.5-0.5B \
    --lora_model outputs-pt \
    --output_dir merged-pt

In [ ]:
# Stage 2: 在PT模型上做SFT
!python supervised_finetuning.py \
    --model_name_or_path merged-pt \
    --train_file_dir ./data/finetune \
    --validation_file_dir ./data/finetune \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --template_name qwen \
    --bf16 \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --num_train_epochs 3 \
    --learning_rate 2e-5 \
    --output_dir outputs-sft \
    --overwrite_output_dir \
    --target_modules all \
    --lora_rank 8 \
    --gradient_checkpointing True

In [ ]:
# 合并SFT模型
!python merge_peft_adapter.py \
    --base_model merged-pt \
    --lora_model outputs-sft \
    --output_dir merged-sft

### 🎯 方案C: 完整流程（PT + SFT + DPO）

在SFT基础上继续做偏好优化

In [ ]:
# Stage 3: DPO训练
# 需要先完成SFT训练

!python dpo_training.py \
    --model_name_or_path ./merged-sft \
    --template_name qwen \
    --train_file_dir ./data/reward \
    --validation_file_dir ./data/reward \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 1 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 100 \
    --max_steps 200 \
    --eval_steps 20 \
    --save_steps 100 \
    --max_source_length 512 \
    --max_target_length 512 \
    --output_dir outputs-dpo \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype bfloat16 \
    --bf16 True \
    --fp16 False \
    --device_map auto \
    --report_to tensorboard \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache

In [ ]:
# 合并DPO模型
!python merge_peft_adapter.py \
    --base_model merged-sft \
    --lora_model outputs-dpo \
    --output_dir merged-dpo

## 📊 Part 4: 查看训练日志

使用TensorBoard查看训练曲线

In [ ]:
# 加载TensorBoard
%load_ext tensorboard
%tensorboard --logdir outputs-sft/runs

## 🧪 Part 5: 快速测试

训练完成后，快速测试一下模型效果

In [ ]:
# 简单推理测试
!python inference.py \
    --base_model merged-sft \
    --template_name qwen \
    --interactive

In [ ]:
# 或者使用Python代码测试
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "merged-sft"  # 或 merged-dpo

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# 测试
messages = [{"role": "user", "content": "介绍一下人工智能"}]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.7)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Response:", response)

## 💾 Part 6: 保存和下载模型

训练完成后，保存模型以便后续使用

### 方式1: 下载LoRA权重（推荐）

LoRA权重很小（50-500MB），容易下载和传输

In [ ]:
# 查看LoRA权重大小
!du -sh outputs-sft/

In [ ]:
# 打包LoRA权重
!zip -r my-sft-lora.zip outputs-sft/

# 下载（会自动弹出下载窗口）
from google.colab import files
files.download('my-sft-lora.zip')

### 方式2: 保存到Google Drive

In [ ]:
# 挂载Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 复制LoRA权重到Drive
!mkdir -p /content/drive/MyDrive/MyMedicalGPT_models
!cp -r outputs-sft /content/drive/MyDrive/MyMedicalGPT_models/

# 如果训练了DPO，也保存
# !cp -r outputs-dpo /content/drive/MyDrive/MyMedicalGPT_models/

print("✓ 模型已保存到 Google Drive!")

### 方式3: 上传到HuggingFace Hub（可选）

In [ ]:
# 安装huggingface_hub
!pip install -q huggingface_hub

from huggingface_hub import HfApi, login

# 登录HuggingFace（需要token）
# 从 https://huggingface.co/settings/tokens 获取token
login()

# 上传模型
api = HfApi()
api.upload_folder(
    folder_path="outputs-sft",
    repo_id="YOUR-USERNAME/my-medical-model",  # 改成你的用户名
    repo_type="model"
)

print("✓ 模型已上传到 HuggingFace Hub!")
print("访问: https://huggingface.co/YOUR-USERNAME/my-medical-model")

## 📝 Part 7: 在本地使用模型

下载模型后，在本地这样使用：

```bash
# 解压LoRA权重
unzip my-sft-lora.zip

# 推理（会自动下载基座模型）
cd MyMedicalGPT
python inference.py \
    --base_model Qwen/Qwen2.5-0.5B-Instruct \
    --lora_model outputs-sft \
    --template_name qwen \
    --interactive
```

## 🎉 完成！

### 下一步可以做什么？

1. **部署到生产环境**
   - 本地GPU推理
   - 云服务器部署
   - vLLM高性能部署

2. **继续优化**
   - 使用更多数据训练
   - 调整超参数
   - 尝试更大的模型

3. **评估模型**
   - 准备测试集
   - 计算BLEU/ROUGE等指标
   - 人工评估

---

### 📚 参考资源

- [项目README](../README.md)
- [快速开始指南](../QUICKSTART.md)
- [推理技术详解](../INFERENCE_GUIDE.md)

祝训练愉快！🚀